In [1]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten, Input, Lambda
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, Conv1D, MaxPooling1D, LSTM, ConvLSTM2D, GRU, CuDNNLSTM, CuDNNGRU, BatchNormalization, LocallyConnected2D, Permute, TimeDistributed, Bidirectional
from keras.layers import Concatenate, Reshape, Softmax, Conv2DTranspose, Embedding, Multiply
from keras import backend as K
from keras.models import load_model

import numpy as np
import scipy
import scipy.sparse as sp

import matplotlib.pyplot as plt

import random, os, h5py, math, time, glob

import torch
import torch.nn as nn
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F

class IdentityEncoder :
    
    def __init__(self, seq_len, channel_map) :
        self.seq_len = seq_len
        self.n_channels = len(channel_map)
        self.encode_map = channel_map
        self.decode_map = {
            val : key for key, val in channel_map.items()
        }
    
    def encode(self, seq) :
        encoding = np.zeros((self.seq_len, self.n_channels))
        
        for i in range(len(seq)) :
            if seq[i] in self.encode_map :
                channel_ix = self.encode_map[seq[i]]
                encoding[i, channel_ix] = 1.

        return encoding
    
    def encode_inplace(self, seq, encoding) :
        for i in range(len(seq)) :
            if seq[i] in self.encode_map :
                channel_ix = self.encode_map[seq[i]]
                encoding[i, channel_ix] = 1.
    
    def encode_inplace_sparse(self, seq, encoding_mat, row_index) :
        raise NotImplementError()
    
    def decode(self, encoding) :
        seq = ''
    
        for pos in range(0, encoding.shape[0]) :
            argmax_nt = np.argmax(encoding[pos, :])
            max_nt = np.max(encoding[pos, :])
            if max_nt == 1 :
                seq += self.decode_map[argmax_nt]
            else :
                seq += self.decode_map[-1]

        return seq
    
    def decode_sparse(self, encoding_mat, row_index) :
        encoding = np.array(encoding_mat[row_index, :].todense()).reshape(-1, 4)
        return self.decode(encoding)

class NopTransformer :
    
    def __init__(self, n_classes) :
        self.n_classes = n_classes
    
    def transform(self, values) :
        return values
    
    def transform_inplace(self, values, transform) :
        transform[:] = values
    
    def transform_inplace_sparse(self, values, transform_mat, row_index) :
        transform_mat[row_index, :] = np.ravel(values)


Using TensorFlow backend.


In [2]:

class GRUClassifier(nn.Module) :
    
    def __init__(self, batch_size) :
        super(GRUClassifier, self).__init__()
        
        hid_numpy = np.zeros((1, batch_size, 64))
        self.hid = Variable(torch.FloatTensor(hid_numpy))
        
        self.gru1_forward = nn.GRU(20, 64, bidirectional=False, num_layers=1, dropout=0.0)
        self.gru1_backward = nn.GRU(20, 64, bidirectional=False, num_layers=1, dropout=0.0)
        self.drop1 = nn.Dropout(p=0.25)
        
        self.fc1 = nn.Linear(in_features=256, out_features=128)
        self.fc2 = nn.Linear(in_features=128, out_features=1)
        
        self.batch_size = batch_size
        self.use_cuda = True if torch.cuda.is_available() else False
        
    def forward(self, x):
        
        #x.shape = (batch_size, 20, 1, 162)
        x = x[:, :, 0, :].transpose(0, 1).transpose(0, 2)
        #x.shape = (162, batch_size, 20)
        x1 = x[:81, :, :]
        x2 = x[81:, :, :]
        
        x1_forward = self.drop1(self.gru1_forward(x1, self.hid)[0])
        x2_forward = self.drop1(self.gru1_forward(x2, self.hid)[0])
        
        x1_flipped = torch.flip(x1, (0,))
        x2_flipped = torch.flip(x2, (0,))
        
        x1_backward = self.drop1(self.gru1_backward(x1_flipped, self.hid)[0])
        x2_backward = self.drop1(self.gru1_backward(x2_flipped, self.hid)[0])
        
        #(seq_len, batch, num_directions * hidden_size)
        x1_forward = x1_forward[-1, ...]
        x2_forward = x2_forward[-1, ...]
        
        x1_backward = x1_backward[-1, ...]
        x2_backward = x2_backward[-1, ...]
        
        x = torch.cat([x1_forward, x1_backward, x2_forward, x2_backward], dim=1)
        
        x = F.relu(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))
        
        return x


In [3]:
#Load pytorch model skeleton

model_pytorch = GRUClassifier(batch_size=1)


In [3]:
#Load predictor

def get_shared_model() :
    
    gru_1 = Bidirectional(GRU(64, activation='tanh', recurrent_activation='sigmoid', recurrent_dropout=0, unroll=False, use_bias=True, reset_after=True, return_sequences=False), merge_mode='concat', name='saved_bidir_1')
    
    drop_1 = Dropout(0.25)

    def shared_model(inp) :

        gru_1_out = gru_1(inp)
        
        drop_1_out = drop_1(gru_1_out)

        return drop_1_out
    
    return shared_model

shared_model = get_shared_model()

#Inputs
res_both = Input(shape=(1, 81 * 2, 19 + 1))

[res_1, res_2] = Lambda(lambda x: [x[:, 0, :81, :], x[:, 0, 81:, :]])(res_both)

#Outputs
true_interacts = Input(shape=(1,))

#Interaction model definition
dense_out_1 = shared_model(res_1)
dense_out_2 = shared_model(res_2)

layer_dense_pair_1 = Dense(128, activation='relu', name='saved_dense_1')
dense_out_pair = layer_dense_pair_1(Concatenate(axis=-1)([dense_out_1, dense_out_2]))

pred_interacts = Dense(1, activation='linear', kernel_initializer='zeros', name='saved_dense_2')(dense_out_pair)
pred_interacts_sigm = Activation('sigmoid')(pred_interacts)

saved_predictor = Model(
    inputs=[
        res_both
    ],
    outputs=pred_interacts_sigm
)

saved_predictor.load_weights('saved_models/ppi_rnn_baker_big_set_5x_negatives_classifier_symmetric_drop_25_5x_negatives_balanced_partitioned_data_epoch_10.h5', by_name=False)
saved_predictor.trainable = False

saved_predictor.compile(
    optimizer=keras.optimizers.SGD(lr=0.1),
    loss='mean_squared_error'
)


In [4]:
saved_predictor.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 1, 162, 20)   0                                            
__________________________________________________________________________________________________
lambda_1 (Lambda)               [(None, 81, 20), (No 0           input_1[0][0]                    
__________________________________________________________________________________________________
saved_bidir_1 (Bidirectional)   (None, 128)          33024       lambda_1[0][0]                   
                                                                 lambda_1[0][1]                   
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 128)          0           saved_bidir_1[0][0]        

In [6]:
#Collect weights from keras model

(
    shuffled_gru_1_kernel_weight_forward,
    shuffled_gru_1_recurrent_weight_forward,
    shuffled_gru_1_bias_bundle_forward,
    shuffled_gru_1_kernel_weight_backward,
    shuffled_gru_1_recurrent_weight_backward,
    shuffled_gru_1_bias_bundle_backward
) = saved_predictor.get_layer('saved_bidir_1').get_weights()

shuffled_gru_1_kernel_weight_forward = shuffled_gru_1_kernel_weight_forward.T
shuffled_gru_1_recurrent_weight_forward = shuffled_gru_1_recurrent_weight_forward.T
shuffled_gru_1_kernel_bias_forward = shuffled_gru_1_bias_bundle_forward[0, :]
shuffled_gru_1_recurrent_bias_forward = shuffled_gru_1_bias_bundle_forward[1, :]

shuffled_gru_1_kernel_weight_backward = shuffled_gru_1_kernel_weight_backward.T
shuffled_gru_1_recurrent_weight_backward = shuffled_gru_1_recurrent_weight_backward.T
shuffled_gru_1_kernel_bias_backward = shuffled_gru_1_bias_bundle_backward[0, :]
shuffled_gru_1_recurrent_bias_backward = shuffled_gru_1_bias_bundle_backward[1, :]

dense_1_weight, dense_1_bias = saved_predictor.get_layer('saved_dense_1').get_weights()
dense_iso_weight, dense_iso_bias = saved_predictor.get_layer('saved_dense_2').get_weights()


In [7]:
#Forward weights

gru_1_kernel_weight_forward = np.concatenate([
    shuffled_gru_1_kernel_weight_forward[64:128, :],
    shuffled_gru_1_kernel_weight_forward[0:64, :],
    shuffled_gru_1_kernel_weight_forward[128:192, :]
], axis=0)

gru_1_recurrent_weight_forward = np.concatenate([
    shuffled_gru_1_recurrent_weight_forward[64:128, :],
    shuffled_gru_1_recurrent_weight_forward[0:64, :],
    shuffled_gru_1_recurrent_weight_forward[128:192, :]
], axis=0)

gru_1_kernel_bias_forward = np.concatenate([
    shuffled_gru_1_kernel_bias_forward[64:128],
    shuffled_gru_1_kernel_bias_forward[0:64],
    shuffled_gru_1_kernel_bias_forward[128:192]
], axis=0)

gru_1_recurrent_bias_forward = np.concatenate([
    shuffled_gru_1_recurrent_bias_forward[64:128],
    shuffled_gru_1_recurrent_bias_forward[0:64],
    shuffled_gru_1_recurrent_bias_forward[128:192]
], axis=0)

#Backward weights

gru_1_kernel_weight_backward = np.concatenate([
    shuffled_gru_1_kernel_weight_backward[64:128, :],
    shuffled_gru_1_kernel_weight_backward[0:64, :],
    shuffled_gru_1_kernel_weight_backward[128:192, :]
], axis=0)

gru_1_recurrent_weight_backward = np.concatenate([
    shuffled_gru_1_recurrent_weight_backward[64:128, :],
    shuffled_gru_1_recurrent_weight_backward[0:64, :],
    shuffled_gru_1_recurrent_weight_backward[128:192, :]
], axis=0)

gru_1_kernel_bias_backward = np.concatenate([
    shuffled_gru_1_kernel_bias_backward[64:128],
    shuffled_gru_1_kernel_bias_backward[0:64],
    shuffled_gru_1_kernel_bias_backward[128:192]
], axis=0)

gru_1_recurrent_bias_backward = np.concatenate([
    shuffled_gru_1_recurrent_bias_backward[64:128],
    shuffled_gru_1_recurrent_bias_backward[0:64],
    shuffled_gru_1_recurrent_bias_backward[128:192]
], axis=0)


In [8]:
#Manually transfer model weights from keras to pytorch

with torch.no_grad() :
    
    model_pytorch.gru1_forward.weight_ih_l0 = nn.Parameter(torch.FloatTensor(gru_1_kernel_weight_forward))
    model_pytorch.gru1_forward.weight_hh_l0 = nn.Parameter(torch.FloatTensor(gru_1_recurrent_weight_forward))
    model_pytorch.gru1_forward.bias_ih_l0 = nn.Parameter(torch.FloatTensor(gru_1_kernel_bias_forward))
    model_pytorch.gru1_forward.bias_hh_l0 = nn.Parameter(torch.FloatTensor(gru_1_recurrent_bias_forward))

    model_pytorch.gru1_backward.weight_ih_l0 = nn.Parameter(torch.FloatTensor(gru_1_kernel_weight_backward))
    model_pytorch.gru1_backward.weight_hh_l0 = nn.Parameter(torch.FloatTensor(gru_1_recurrent_weight_backward))
    model_pytorch.gru1_backward.bias_ih_l0 = nn.Parameter(torch.FloatTensor(gru_1_kernel_bias_backward))
    model_pytorch.gru1_backward.bias_hh_l0 = nn.Parameter(torch.FloatTensor(gru_1_recurrent_bias_backward))
    
    model_pytorch.fc1.weight = nn.Parameter(torch.FloatTensor(np.transpose(dense_1_weight, (1, 0))))
    model_pytorch.fc1.bias = nn.Parameter(torch.FloatTensor(dense_1_bias))
    
    model_pytorch.fc2.weight = nn.Parameter(torch.FloatTensor(np.transpose(dense_iso_weight, (1, 0))))
    model_pytorch.fc2.bias = nn.Parameter(torch.FloatTensor(dense_iso_bias))


In [9]:
#Save pytorch model

torch.save(model_pytorch.state_dict(), "saved_models/ppi_rnn_baker_big_set_5x_negatives_classifier_symmetric_drop_25_5x_negatives_balanced_partitioned_data_epoch_10_pytorch.pth")


In [5]:
#Initialize sequence encoder

seq_length = 81

residue_map = {'D': 0, 'E': 1, 'V': 2, 'K': 3, 'R': 4, 'L': 5, 'S': 6, 'T': 7, 'N': 8, 'H': 9, 'A': 10, 'I': 11, 'G': 12, 'P': 13, 'Q': 14, 'Y': 15, 'W': 16, 'M': 17, 'F': 18, '#': 19}

encoder = IdentityEncoder(seq_length, residue_map)


In [6]:
#Load pytorch model

model_pytorch = GRUClassifier(batch_size=1)

_ = model_pytorch.load_state_dict(torch.load("saved_models/ppi_rnn_baker_big_set_5x_negatives_classifier_symmetric_drop_25_5x_negatives_balanced_partitioned_data_epoch_10_pytorch.pth"))


In [7]:
#Binder DHD_154

batch_size = 1

seq_1 = "TAEELLEVHKKSDRVTKEHLRVSEEILKVVEVLTRGEVSSEVLKRVLRKLEELTDKLRRVTEEQRRVVEKLN"[:81]
seq_2 = "DLEDLLRRLRRLVDEQRRLVEELERVSRRLEKAVRDNEDERELARLSREHSDIQDKHDKLAREILEVLKRLLERTE"[:81]

print("Seq 1 = " + seq_1)
print("Seq 2 = " + seq_2)

encoder = IdentityEncoder(81, residue_map)

test_onehot_1 = np.tile(np.expand_dims(np.expand_dims(encoder.encode(seq_1), axis=0), axis=0), (batch_size, 1, 1, 1))
test_onehot_2 = np.tile(np.expand_dims(np.expand_dims(encoder.encode(seq_2), axis=0), axis=0), (batch_size, 1, 1, 1))

test_len_1 = np.tile(np.array([[len(seq_1)]]), (batch_size, 1))
test_len_2 = np.tile(np.array([[len(seq_2)]]), (batch_size, 1))


Seq 1 = TAEELLEVHKKSDRVTKEHLRVSEEILKVVEVLTRGEVSSEVLKRVLRKLEELTDKLRRVTEEQRRVVEKLN
Seq 2 = DLEDLLRRLRRLVDEQRRLVEELERVSRRLEKAVRDNEDERELARLSREHSDIQDKHDKLAREILEVLKRLLERTE


In [8]:
#Predict using keras model

y_pred_keras = saved_predictor.predict(x=[np.concatenate([test_onehot_1, test_onehot_2], axis=2)], batch_size=1)

print("y_pred (keras)   = " + str(y_pred_keras[0]))

#Predict using pytorch model
model_pytorch.eval()
        
input_var = Variable(torch.FloatTensor(np.transpose(np.concatenate([test_onehot_1, test_onehot_2], axis=2), (0, 3, 1, 2))))
input_var = input_var.cuda() if model_pytorch.use_cuda else input_var

y_pred_pytorch = model_pytorch(input_var).data.cpu().numpy()

print("y_pred (pytorch) = " + str(y_pred_pytorch[0]))


y_pred (keras)   = [0.853256]
y_pred (pytorch) = [0.8532558]
